In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("data source").getOrCreate()

In [3]:
spark

In [1]:
# from spylon_kernel import register_ipython_magics
# register_ipython_magics()

In [1]:
# %%scala 
# val x=8
# x

UsageError: Cell magic `%%scala` not found.


In [4]:
df = spark.read.format("csv")\
    .option("header","true") \
    .option("inferschema","true")\
    .load("./data/retail-data/all/*.csv")



In [5]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



## Data Sources

- CSV
- JSON
- Parquet
- ORC
- JDBC/ODBC connections
- Plain-text files

`community-created`
- Cassandra
- HBase
- MangoDB
- AWS Redshift
- XML

## mode
`read mode` 
- 1. permissive -> null for currupted records and place in _currupt_record
- 2. dropMalformed -> drop the row contains malformed records
- 3. failFast -> fail immediately upon malformed records

`write mode`
* 1. append
* 2. overwrite
* 3. errorifExist
* 4. ignore

In [8]:
# DataFrameReader.format(...).option("key", "value").schema(...).load()
# DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()


In [9]:
df = spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .option("mode","FAILFAST")\
    .load("./data/flight-data/csv/2010-summary.csv")

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [13]:
df.write.format("csv").option("sep","\t").save("file.csv")

In [11]:
df.count()

255

In [6]:
df.repartition(5).write.save("./output/repart_flightdata")

In [16]:
# json 
df_json = spark.read.format("json")\
    .option('inferSchema',"true")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .load("./data/flight-data/json/2010-summary.json")

In [17]:
df_json.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [18]:
df_json.write.format("json").option("mode","append").save("./output/flight.json")

In [20]:
# parquet

spark.read.format("parquet")\
    .load("./data/flight-data/parquet/2010-summary.parquet/").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [23]:
df.write.format("parquet").mode("overwrite").save("./output/flight_data.parquet")

In [21]:
# orc
spark.read.format("orc")\
    .load("./data/flight-data/orc/2010-summary.orc/").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [22]:
df.write.format("orc").mode("overwrite").save("./output/orc_file.orc")

In [50]:
# jdbc
## sqlLite
star = "C:/Users/Avinash Godbole/Desktop/Programm File/pyspark/oriily_book2018"
driver = "org.sqlite.JDBC"
path = "//data/flight-data/jdbc/my-sqlite.db"
url = "jdbc:sqlite:"+path
tablename = "flight_info"

df_lite = spark.read.format("jdbc")\
    .option("url",url)\
    .option("dbtable",tablename)\
    .option("driver",driver).load()

Py4JJavaError: An error occurred while calling o243.load.
: java.lang.ClassNotFoundException: org.sqlite.JDBC
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:35)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql

In [38]:
## mysql
driver = "com.mysql.jdbc.Driver"
username="root"
password="root"
url= "jdbc:mysql://localhost"
dbtable="pyspark.flightData1"

df_jdbc = spark.read.format("jdbc")\
    .option("driver",driver)\
    .option("url",url)\
    .option("user",username)\
    .option("password",password)\
    .option("dbtable",dbtable)\
    .load()

In [33]:
df.write.format("jdbc")\
    .option("driver",driver)\
    .option("url",url)\
    .option("user",username)\
    .option("password",password)\
    .option("dbtable",dbtable)\
    .save()

In [48]:
# in Python
pushdownQuery = """(SELECT DISTINCT(DEST_COUNTRY_NAME) FROM pyspark.flightdata) as a"""
dbDataFrame = spark.read.format("jdbc")\
    .option("url", url)\
    .option("user",username)\
    .option("password",password)\
    .option("dbtable", pushdownQuery)\
    .option("driver", driver)\
    .load()

In [49]:
dbDataFrame.show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|               Egypt|
|   Equatorial Guinea|
|          Costa Rica|
|             Senegal|
|              Guyana|
|               Malta|
|             Bolivia|
|            Anguilla|
|Turks and Caicos ...|
|Saint Vincent and...|
|               Italy|
|            Pakistan|
|             Iceland|
|    Marshall Islands|
|          Luxembourg|
|            Honduras|
|         The Bahamas|
|         El Salvador|
|               Samoa|
+--------------------+
only showing top 20 rows



In [31]:
# write to mysql
driver = "com.mysql.jdbc.Driver"
url = "jdbc:mysql://localhost"
username= 'root'
password = 'root'
dbtable='pyspark.flightData'

df.write.format('jdbc').options(url=url,driver=driver,dbtable=dbtable,user=username,password=password)\
    .save()

In [35]:
df_jdbc.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [1]:
# partition from mysql 
porper = {
    
    "username": 'root',
    "password" : 'root'
}

url = "jdbc:mysql://localhost"
tablename = 'pyspark.flightdata'
colName= "count"
lowerBound = 0
upperBound = 34113
numPartitions = 10

spark.read.jdbc(url=url,table=tablename,column=colName,properties=porper,lowerBound=lowerBound,upperBound=upperBound,numPartitions=10).count()

NameError: name 'spark' is not defined

In [73]:
spark.read.jdbc(url,table="pyspark.flightdata",properties=porper)

Py4JJavaError: An error occurred while calling o312.jdbc.
: java.sql.SQLException: Access denied for user 'Avinash Godbole'@'localhost' (using password: YES)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:833)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:453)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverWrapper.connect(DriverWrapper.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:67)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:56)
	at org.apache.spark.sql.execution.dataso

In [13]:
# partitioning
df.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
    .save("./output/partitioned_file.parquet")

In [10]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [12]:
df.select('DEST_COUNTRY_NAME').distinct().count()

125

In [15]:
# bucket hive Bucketing is supported only for Spark-managed tables.
no_bucket = 10
column_bucket = "count"

df.write.mode("overwrite")\
    .bucketBy(10,"count").save("bucketFiles")

AnalysisException: 'save' does not support bucketBy right now;

################# end ###################

%%scala 

val df = spark.read.format("csv").load("./data/retail-data/all/online-retail-dataset.csv")

%%scala 
spark.stop

In [2]:
# read data from file add column current data and save to mysql
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col,current_date,lit

if __name__ == "__main__":
    # spark session
    spark = SparkSession.builder.appName("data move").getOrCreate()

    # read data from csv 
    print("reading data from csv")
    df = spark.read.format("csv")\
        .option("header","true")\
        .option("inferSchema","true")\
        .option("mode","FAILFAST")\
        .load("./data/flight-data/csv/2010-summary.csv")

    df2 = df.withColumn("date",lit(current_date()))


    driver = "com.mysql.jdbc.Driver"
    url = "jdbc:mysql://localhost"
    username= 'root'
    password = 'root'
    dbtable='pyspark.flightdate'


    # savinng to mysql
    print("saving to mysql...")
    df2.write.format("jdbc")\
        .option("driver",driver)\
        .option("url",url)\
        .option("user",username)\
        .option("password",password)\
        .option("dbtable",dbtable)\
        .save()

    print(f"succesfulley saved in mysql with name {dbtable}")

In [16]:
# "com.mysql.cj.jdbc.Driver" new driver

In [17]:
spark.stop()